In [3]:
import os
import cv2
import mediapipe as mp
import numpy as np

In [26]:
#vid_01 = cv2.VideoCapture('/media/bernasconi/My Passport/MUSIC/VID_20211211_110947.mp4')
HANDS = '/media/bernasconi/My Passport/MUSIC/VID_20211211_110947.mp4'
FACE = '/media/bernasconi/My Passport/MUSIC/VID_20211211_115747.mp4'
BODY = '/media/bernasconi/My Passport/MUSIC/VID_20211211_115354.mp4'
HANDS_02 = '/media/bernasconi/My Passport/MUSIC/VID_20211211_111749.mp4'
BODY_02 = '/media/bernasconi/My Passport/MUSIC/VID_20211211_115533.mp4'

In [22]:
def process_video(video_path, body=False, hand=False, face=False):
    
    filename = os.path.splitext(os.path.basename(video_path))[0]
    dirname = os.path.dirname(video_path)
    
    savepath = dirname+"/material_"+filename
    
    try:
        os.mkdir(savepath)
    except OSError as error:
        pass
    
    #read video
    vid_01 = cv2.VideoCapture(video_path)
    cv2.namedWindow("output", cv2.WINDOW_NORMAL)
    
    #create mediapipe tool
    if face:
        mp_face_mesh = mp.solutions.face_mesh
        face =  mp_face_mesh.FaceMesh(max_num_faces=1,
                                      min_detection_confidence=0,
                                      min_tracking_confidence=0)
    elif hand:
        mpHands = mp.solutions.hands
        hands = mpHands.Hands(max_num_hands=2)
    else:
        mpPose = mp.solutions.pose
        pose = mpPose.Pose(
            min_detection_confidence=0)
        
    mpDraw = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    
    #store kp
    kps = []
    kp = []
    
    i = 0
    
    images = []
    
    while(vid_01.isOpened()):
        ret, frame = vid_01.read()
        if ret == True:
            
            imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            h, w, c = frame.shape
            
            empty_frame = np.zeros([h,w,3],dtype=np.uint8)
            empty_frame.fill(255)
                
            if face:
                results = face.process(imgRGB)
                if results.multi_face_landmarks:
                    for faceLms in results.multi_face_landmarks:
                        kp = []
                        for id_, lm in enumerate(faceLms.landmark):
                            kp.append([lm.x, lm.y])
                            
                        mpDraw.draw_landmarks(empty_frame, faceLms, mp_face_mesh.FACEMESH_TESSELATION,
                            landmark_drawing_spec=None, 
                            connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
                        
                        mpDraw.draw_landmarks(empty_frame, faceLms, mp_face_mesh.FACEMESH_CONTOURS,
                            landmark_drawing_spec=None, 
                            connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
                    
                        kps.append(kp)
                
            elif hand:
                results = hands.process(imgRGB)
                if results.multi_hand_landmarks:
                    for handLms in results.multi_hand_landmarks:
                        kp = []
                        kp_bbox = []
                        for id_, lm in enumerate(handLms.landmark):
                            kp.append([lm.x,lm.y])
                            cx, cy = int(lm.x*w), int(lm.y*h)
                            kp_bbox.append([cx, cy])

                        mpDraw.draw_landmarks(empty_frame, handLms, mpHands.HAND_CONNECTIONS,
                            mpDraw.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=2),
                            mpDraw.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=2))
             

                        kps.append(kp)
            else:
                results = pose.process(imgRGB)
                if results.pose_landmarks:
                    for poseLms in results.pose_landmarks.landmark:
                        kp.append([poseLms.x, poseLms.y])
                
                    mpDraw.draw_landmarks(
                        empty_frame,
                        results.pose_landmarks,
                        mpPose.POSE_CONNECTIONS,
                        mpDraw.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=2)) 
    
                    kps.append(kp)
            
            frame_name = filename +'_frame_'+str(i)+'.jpg'
            frame_img_path = savepath+'/'+frame_name
            cv2.imwrite(frame_img_path, empty_frame) 
            images.append(empty_frame)
            
            i+=1
            cv2.imshow("output", empty_frame)
            k = cv2.waitKey(20)
            # 113 is ASCII code for q key
            if k == 113:
                break
        else:
            break
    vid_01.release()
    cv2.destroyAllWindows()
    
    #store keypoints
    textfile = open(savepath+"/keypoints.txt", "w")
    for element in kps:
        textfile.write('\n'.join(str(kp_) for kp_ in element))
    textfile.close()
    
    #store video
    video = cv2.VideoWriter(savepath+"/kps_video.avi", 0, 24, (w, h))
    for image in images:
        video.write(image)

    
    return kps

In [28]:
kps = process_video(HANDS_02, hand=True)